<center>
    <p style="text-align:center">
        <img alt="phoenix logo" src="https://storage.googleapis.com/arize-phoenix-assets/assets/phoenix-logo-light.svg" width="200"/>
        <br>
        <a href="https://docs.arize.com/phoenix/">Docs</a>
        |
        <a href="https://github.com/Arize-ai/phoenix">GitHub</a>
        |
        <a href="https://arize-ai.slack.com/join/shared_invite/zt-2w57bhem8-hq24MB6u7yE_ZF_ilOYSBw#/shared-invite/email">Community</a>
    </p>
</center>

# Google GenAI SDK - Building a Routing Agent

## Install Dependencies

In [ ]:
!pip install -q google-genai arize-phoenix-otel openinference-instrumentation-google-genai

## Connect to Arize Phoenix

In [ ]:
import os
from getpass import getpass

from google import genai

from phoenix.otel import register

if "PHOENIX_API_KEY" not in os.environ:
    os.environ["PHOENIX_API_KEY"] = getpass("Enter your Phoenix API key: ")

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com/"

tracer_provider = register(auto_instrument=False, project_name="google-genai-routing-agent")
tracer = tracer_provider.get_tracer(__name__)

## Authenticate with Google Vertex AI

In [ ]:
!gcloud auth login

In [ ]:
from google.genai import types

In [ ]:
# Create a client using the Vertex AI API, you could also use the Google GenAI API instead here
client = genai.Client(vertexai=True, project="<ADD YOUR GCP PROJECT ID>", location="us-central1")

# Routing

In [ ]:
@tracer.tool()
def get_current_weather(location: str) -> str:
    """Returns the current weather.

    Args:
      location: The city and state, e.g. San Francisco, CA
    """
    return "sunny"


@tracer.tool()
def get_current_time(location: str) -> str:
    """Returns the current time.

    Args:
      location: The city and state, e.g. San Francisco, CA
    """
    return "10:00 AM"


@tracer.llm()
def call_google_genai(query: str):
    response = client.models.generate_content(
        model="gemini-2.0-flash-001",
        contents=query,
        config=types.GenerateContentConfig(tools=[get_current_weather, get_current_time]),
    )
    return response.text


print(call_google_genai(query="What is the weather like in Boston?"))
print(call_google_genai(query="What is the time in Boston?"))